In [ ]:
class Student:
    def __init__(self, student_id, student_name):
        self.student_id = student_id
        self.student_name = student_name
        self.registered_courses = []

    def register_course(self, course):
        if course not in self.registered_courses:
            self.registered_courses.append(course)
        
    def drop_course(self, course):
        if course in self.registered_courses:
            self.registered_courses.remove(course)

class BSTNode:
    def __init__(self, course):
        self.course = course
        self.left = None
        self.right = None

class BST:
    def __init__(self):
        self.root = None

    def insert(self, course):
        if self.root is None:
            self.root = BSTNode(course)
        else:
            self._insert(self.root, course)

    def _insert(self, node, course):
        if course.course_code < node.course.course_code:
            if node.left:
                self._insert(node.left, course)
            else:
                node.left = BSTNode(course)
        else:
            if node.right:
                self._insert(node.right, course)
            else:
                node.right = BSTNode(course)

    def search(self, course_code):
        return self._search(self.root, course_code)

    def _search(self, node, course_code):
        if not node or node.course.course_code == course_code:
            return node.course if node else None
        if course_code < node.course.course_code:
            return self._search(node.left, course_code)
        return self._search(node.right, course_code)

class QueueNode:
    def __init__(self, student):
        self.student = student
        self.next = None

class Queue:
    def __init__(self):
        self.front = None
        self.rear = None

    def insert(self, student):
        new_node = QueueNode(student)
        if self.rear is None:
            self.front = self.rear = new_node
        else:
            self.rear.next = new_node
            self.rear = new_node

    def delete(self):
        if self.front is None:
            return None
        student = self.front.student
        self.front = self.front.next
        if self.front is None:
            self.rear = None
        return student

    def is_empty(self):
        return self.front is None

class Graph:
    def __init__(self):
        self.adj_list = {}

    def add_course(self, course_code):
        if course_code not in self.adj_list:
            self.adj_list[course_code] = []

    def add_prerequisite(self, course_code, prereq_code):
        if course_code in self.adj_list:
            self.adj_list[course_code].append(prereq_code)

    def get_prerequisites(self, course_code):
        return self.adj_list.get(course_code, [])

class Course:
    def __init__(self, course_code, course_name, max_capacity):
        self.course_code = course_code
        self.course_name = course_name
        self.max_capacity = max_capacity
        self.enrolled_students = []
        self.waitlist = Queue()

    def enroll_student(self, student):
        if len(self.enrolled_students) < self.max_capacity:
            self.enrolled_students.append(student)
            return True
        else:
            self.waitlist.insert(student)
            return False

    def drop_student(self, student):
        if student in self.enrolled_students:
            self.enrolled_students.remove(student)
            if not self.waitlist.is_empty():
                next_student = self.waitlist.delete()
                self.enrolled_students.append(next_student)

class CourseRegistrationSystem:
    def __init__(self):
        self.students = {}
        self.course_catalog = BST()
        self.prerequisite_graph = Graph()

    def add_student(self):
        while True:
            student_id = input("Enter Student ID: ")
            if not student_id.isdigit():
                print("Invalid Student ID! Please enter numbers only.")
                continue
            student_name = input("Enter Full Name: ")
            if not student_name.replace(" ", "").isalpha():
                print("Invalid Name! Please enter alphabets only.")
                continue
            student = Student(student_id, student_name)
            self.students[student_id] = student
            print(f"Student {student_name} added successfully!")
            break


    def add_course(self):
        while True:
            course_code = input("Enter Course Code: ")
            if not course_code.isalnum():
                print("Invalid Course Code! Please enter only letters and numbers.")
                continue

            course_name = input("Enter Course Name: ")
            if not course_name.replace(" ", "").isalpha():
                print("Invalid Course Name! Please enter alphabets only.")
                continue

            max_capacity_input = input("Enter Maximum Capacity: ")
            if not max_capacity_input.isdigit() or int(max_capacity_input) <= 0:
                print("Invalid Capacity! Please enter a positive number.")
                continue

            max_capacity = int(max_capacity_input)
            course = Course(course_code, course_name, max_capacity)
            self.course_catalog.insert(course)
            self.prerequisite_graph.add_course(course_code)

        # Ask for prerequisites
            prereqs = input("Enter prerequisite course codes (comma-separated, or press Enter for none): ")
            if prereqs:
                prereq_list = [code.strip() for code in prereqs.split(",")]
                for prereq_code in prereq_list:
                    self.prerequisite_graph.add_prerequisite(course_code, prereq_code)

            print(f"Course {course_name} ({course_code}) added successfully with prerequisites: {prereqs if prereqs else 'None'}")
            break


    def register_student_for_course(self):
        student_id = input("Enter Student ID: ")
        course_code = input("Enter Course Code: ")
        student = self.students.get(student_id)
        course = self.course_catalog.search(course_code)
        
        if not student or not course:
            print("Student or Course not found!")
            return

        unmet_prereqs = [p for p in self.prerequisite_graph.get_prerequisites(course_code) if p not in [c.course_code for c in student.registered_courses]]
        if unmet_prereqs:
            print(f"Cannot register! Missing prerequisites: {', '.join(unmet_prereqs)}")
            return

        if course.enroll_student(student):
            student.register_course(course)
            print(f"{student.student_name} successfully registered for {course.course_name}.")
        else:
            print(f"Course full! {student.student_name} added to waitlist.")

    def view_all_courses(self):
        def in_order_traversal(node):
            if node:
                in_order_traversal(node.left)
                print(f"{node.course.course_name} ({node.course.course_code})")
                in_order_traversal(node.right)
        in_order_traversal(self.course_catalog.root)

def main():
    system = CourseRegistrationSystem()

    while True:
        print("\n Student Registration System ")
        print("1. Add a Student")
        print("2. Add a Course")
        print("3. Register Student for Course")
        print("4. View All Courses")
        print("5. Exit")
        
        choice = input("Enter your choice: ")

        if choice == '1':
            system.add_student()
        elif choice == '2':
            system.add_course()
        elif choice == '3':
            system.register_student_for_course()
        elif choice == '4':
            system.view_all_courses()
        elif choice == '5':
            print("Thank you for registering!")
            break
        else:
            print("Invalid choice. Try again.")

if __name__ == "__main__":
    main()



 Student Registration System 
1. Add a Student
2. Add a Course
3. Register Student for Course
4. View All Courses
5. Exit


Enter your choice:  1
Enter Student ID:  24110
Enter Full Name:  Diya Dineep


Student Diya Dineep added successfully!

 Student Registration System 
1. Add a Student
2. Add a Course
3. Register Student for Course
4. View All Courses
5. Exit


Enter your choice:  1
Enter Student ID:  24157
Enter Full Name:  V Monisha Reddy


Student V Monisha Reddy added successfully!

 Student Registration System 
1. Add a Student
2. Add a Course
3. Register Student for Course
4. View All Courses
5. Exit


Enter your choice:  2
Enter Course Code:  22AIE112
